In [ ]:
# library(edgeR)
library(Seurat)
library(tidyr)
# library(ggplot2)
library(patchwork)
library(dplyr)
library(cowplot)
# library(future)
# library(DoubletFinder)
library(RColorBrewer)
library(harmony)
library(psych)
library(corrplot)
colors = c(
  "#1B9E77", "#80B1D3", "#BEAED4", "#FB8072", "#386CB0", "#F0027F",
  "#E78AC3", "#BF5B17", "#7FC97F", "#FF7F00", "#7570B3", "#E6AB02", 
  "#FF0000", "#A6761D", "#1F78B4", "#B2DF8A", "#33A02C", "#FB9A99",
  "#E31A1C", "#FDBF6F", "#E5D8BD", "#CAB2D6", "#6A3D9A", "#B15928",
  "#FBB4AE", "#B3CDE3", "#BC80BD", "#CCEBC5", "#DECBE4", "#FED9A6",
  "#FFFFB3", "#FDDAEC", "#B3E2CD", "#FDCDAC", "#CBD5E8", "#F4CAE4",
  "#E6F5C9", "#FFF2AE", "#F1E2CC", "#E41A1C", "#377EB8", "#984EA3",
  "#D95F02", "#F781BF", "#FFED6F", "#66C2A5", "#FC8D62", "#8DA0CB", 
  "#A6D854", "#FFD92F", "#E5C494", "#8DD3C7", "#A65628", "#FCCDE5",
  "#BEBADA", "#FDC086", "#A6CEE3", "#FDB462")

In [ ]:
human <- readRDS("/data/work/Brain/project/RNA/human.rds")
mouse <- readRDS("/data/work/Brain/project/RNA/mouse.rds")
mac <- readRDS("/data/work/Brain/project/RNA/macaque.rds")

human_meta = human@meta.data
mm_meta = mouse@meta.data
mac_meta = mac@meta.data

com_metadata <- rbind(human_meta,mm_meta,mac_meta)

human_counts=human@assays$RNA@counts[human_gene,]
mouse_counts=mouse[['RNA']]$counts[mouse_gene,]
mac_counts = mac@assays$RNA@counts[mac_gene,] # noticed that gene name of three count matrix should be same

In [ ]:
integrated <- CreateSeuratObject(counts = list(
    human = human_counts,
    macaque = mac_counts,
    mouse = mouse_counts),
    meta.data = com_metadata
)

integrated <- NormalizeData(integrated)
integrated <- FindVariableFeatures(integrated)
integrated <- ScaleData(integrated)
integrated <- RunPCA(integrated)

integrated <- IntegrateLayers(object = integrated, method = CCAIntegration, orig.reduction = "pca", new.reduction = "integrated.cca",
    verbose = FALSE)

# re-join layers after integration
integrated[["RNA"]] <- JoinLayers(integrated[["RNA"]])

integrated <- FindNeighbors(integrated, reduction = "integrated.cca", dims = 1:30)
integrated <- FindClusters(integrated, resolution = 0.8)
integrated <- RunUMAP(integrated, dims = 1:30, reduction = "integrated.cca")

In [ ]:
saveRDS(integrated,"/data/work/Brain/project/RNA/integrated.rds")